In [1]:
from SpacetimeVar import *

In [2]:
dim = 3
path = "../Source/HamCom/"

In [3]:
fileRHS = open(path+"ET_Integration_Rhs_K.H", "w+")
fileRHS.write("#ifndef ET_INTEGRATION_RHS_K_H \n")
fileRHS.write("#define ET_INTEGRATION_RHS_K_H \n\n")

fileRHS.write("#include <AMReX_REAL.H> \n")
fileRHS.write("#include <AMReX_Array4.H> \n")
fileRHS.write("#include <ET_Integration_Setup.H> \n\n")

fileRHS.write("AMREX_GPU_DEVICE \ninline \nvoid \n")
fileRHS.write("state_rhs(int i, int j, int k, \n")
fileRHS.write("        amrex::Array4<amrex::Real> const& rhs_fab, \n")
fileRHS.write("        amrex::Array4<amrex::Real const> const& state_fab, \n")
fileRHS.write("        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx, \n")
fileRHS.write("        const amrex::GeometryData& geom) noexcept \n{\n")
fileRHS.write("        const auto domain_xlo = geom.ProbLo(); \n\n")
fileRHS.write("        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; \n")
fileRHS.write("        amrex::Real x1 = (j + 0.5)*geom.CellSize(0) + domain_xlo[1]; \n")
fileRHS.write("        amrex::Real x2 = (k + 0.5)*geom.CellSize(0) + domain_xlo[2]; \n\n")

71

In [4]:
stVar.decl = []
u = stVar('u', declare = True)

In [5]:
fileRHS.write(u.AMReXDeclare())

52

In [6]:
DiffOrder = 2

In [7]:
fileRHS.write(DstVar(u, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(u, 2, orderD = DiffOrder).AMReXDeclare())

1797

In [8]:
x = stVarRank1('x')
r = stVar('r')
r.var = sp.sqrt(x.symb[0]**2+x.symb[1]**2+x.symb[2]**2)

c1 = stVarRank1('c1')
c2 = stVarRank1('c2')
c1.var = np.array([3.0,0,0])
c2.var = np.array([-3.0,0,0])

P1 = stVarRank1('P1')
P2 = stVarRank1('P2')

P1.var = np.array([0,0.2,0])
P2.var = np.array([0,-0.2,0])

xc1 = stVarRank1('xc1')
xc2 = stVarRank1('xc2')

for i in range(3):
    xc1.var[i] = x.symb[i]-c1.symb[i]
    xc2.var[i] = x.symb[i]-c2.symb[i]

rc1 = stVar('rc1')
rc1.var = sp.sqrt(xc1.symb[0]**2 + xc1.symb[1]**2 + xc1.symb[2]**2)

rc2 = stVar('rc2')
rc2.var = sp.sqrt(xc2.symb[0]**2 + xc2.symb[1]**2 + xc2.symb[2]**2)

eps1 = stVar('eps1')
eps2 = stVar('eps2')

n1 = stVarRank1('n1')
n2 = stVarRank1('n2')

for i in range(3):
    n1.var[i] = xc1.symb[i]/rc1.symb
    n2.var[i] = xc2.symb[i]/rc2.symb

In [9]:
fileRHS.write(r.AMReXReal())
fileRHS.write(c1.AMReXReal())
fileRHS.write(c2.AMReXReal())
fileRHS.write(P1.AMReXReal())
fileRHS.write(P2.AMReXReal())
fileRHS.write(xc1.AMReXReal())
fileRHS.write(xc2.AMReXReal())
fileRHS.write(rc1.AMReXReal())
fileRHS.write(rc2.AMReXReal())
fileRHS.write(n1.AMReXReal())
fileRHS.write(n2.AMReXReal())

108

In [10]:
Atilde1UU = stVarRank2('Atilde1UU')
Atilde2UU = stVarRank2('Atilde2UU')
for i in range(3):
    for j in range(3):
        Atilde1UU.var[i][j] += 3/(2*(rc1.symb+eps1.symb)**2)*(P1.symb[i]*n1.symb[j]+P1.symb[j]*n1.symb[i])
        Atilde2UU.var[i][j] += 3/(2*(rc2.symb+eps2.symb)**2)*(P2.symb[i]*n2.symb[j]+P2.symb[j]*n2.symb[i])
        for k in range(3):
            Atilde1UU.var[i][j] += -3/(2*(rc1.symb+eps1.symb)**2)*(sp.eye(3)[i,j]-n1.symb[i]*n1.symb[j])*n1.symb[k]*P1.symb[k]
            Atilde2UU.var[i][j] += -3/(2*(rc2.symb+eps2.symb)**2)*(sp.eye(3)[i,j]-n2.symb[i]*n2.symb[j])*n2.symb[k]*P2.symb[k]

In [11]:
AtildeUU = stVarRank2('AtildeUU')
for i in range(3):
    for j in range(3):
        AtildeUU.var[i][j] += Atilde1UU.symb[i][j]+Atilde2UU.symb[i][j]

In [12]:
fileRHS.write(Atilde1UU.AMReXReal())
fileRHS.write(Atilde2UU.AMReXReal())
fileRHS.write(AtildeUU.AMReXReal())

540

In [13]:
M1 = stVar('M1')
M2 = stVar('M2')

M1.var = 1.0
M2.var = 1.0

alpha = stVar('alpha')

alpha.var = 1/(M1.symb/(rc1.symb+eps1.symb)+M2.symb/(rc2.symb+eps2.symb))

In [14]:
fileRHS.write(M1.AMReXReal())
fileRHS.write(M2.AMReXReal())
fileRHS.write(alpha.AMReXReal())

69

In [15]:
beta = stVar('beta')

for i in range(3):
    for j in range(3):
        beta.var += 1/8*(alpha.symb**7)*AtildeUU.symb[i][j]*AtildeUU.symb[i][j]

In [16]:
fileRHS.write(beta.AMReXReal())

489

In [17]:
RHS_u = stVar('u')

RHS_u.var = 0
for i in range(3):
    RHS_u.var += Dsymb(u.symb,str(i)+str(i))
    
RHS_u.var += beta.symb*(alpha.symb*(1+u.symb)+1)**(-7)

In [18]:
fileRHS.write(RHS_u.AMReXRHS())

97

In [19]:
fileRHS.write("}\n")
fileRHS.write("#endif")
fileRHS.close()

In [20]:
stVar.declDiag

[]